In [ ]:
import sys  
!{sys.executable} -m pip install matplotlib python-dateutil

In [348]:
import calendar
import datetime as dt
import dateutil.parser as dp
import pandas as pd
import matplotlib.pyplot as plt

In [349]:
df = pd.read_json(r'data.json', convert_dates=True)
df["duration"] = (df["end_time"] - df["start_time"]).dt.days
df["count"] = 1

In [350]:
cancelled = df[(df["idList"] == "Cancelled")]
cancelled

,name,shortUrl,idList,start_time,end_time,duration,count
46,THINKST_CANARY,https://trello.com/c/4t0WMyWX,Cancelled,2020-06-25 15:29:58+00:00,2020-08-07 15:00:00+00:00,42,1
47,CLEARPASS: Fix user.userid,https://trello.com/c/Awr4fAV2,Cancelled,2020-07-06 18:52:35+00:00,2020-07-06 20:34:07+00:00,0,1
48,MICROSOFT_SECURITY_CENTER_ALERT,https://trello.com/c/oKgvXfJ5,Cancelled,2020-06-01 12:37:00+00:00,2020-06-17 15:00:00+00:00,16,1
49,AIRWATCH,https://trello.com/c/QNhbqNMe,Cancelled,2020-04-07 12:37:00+00:00,2020-06-17 15:00:00+00:00,71,1
50,F5_ASM,https://trello.com/c/MkUaoJLR,Cancelled,2020-05-04 15:00:00+00:00,2020-05-15 15:00:00+00:00,11,1
51,AWS_CLOUDTRAIL [ListAccessKeys],https://trello.com/c/fmbF0DfM,Cancelled,2020-03-25 15:00:00+00:00,2020-05-14 15:00:00+00:00,50,1
52,VMWARE_VCENTER,https://trello.com/c/hU1a0G4C,Cancelled,2020-04-28 15:00:00+00:00,2020-05-07 15:00:00+00:00,9,1
53,EXTRAHOP_DHCP,https://trello.com/c/WBf5AakG,Cancelled,2020-04-13 15:00:00+00:00,2020-05-05 15:00:00+00:00,22,1
54,THYCOTIC,https://trello.com/c/UaX8VGRn,Cancelled,2020-04-27 15:00:00+00:00,2020-04-27 15:00:00+00:00,0,1
55,PROOFPOINT_TAP,https://trello.com/c/YnuBoZLD,Cancelled,2020-04-27 15:00:00+00:00,2020-04-28 15:00:00+00:00,1,1


In [351]:
cancelled["duration"].describe()

count    11.000000
mean     21.000000
std      23.405128
min       0.000000
25%       5.000000
50%      11.000000
75%      32.000000
max      71.000000
Name: duration, dtype: float64

In [369]:
done = df[(df["idList"] == "Done") & (df["duration"] > 20) & (df["duration"] <= 30)]
done

,name,shortUrl,idList,start_time,end_time,duration,count
8,IPSWITCH_SFTP,https://trello.com/c/H76wnXs7,Done,2020-07-17 15:12:18+00:00,2020-08-07 19:00:13+00:00,21,1
13,GMAIL_LOGS,https://trello.com/c/TdMY8bpZ,Done,2020-06-23 14:14:01+00:00,2020-07-20 13:33:34+00:00,26,1
15,F5_ASM default,https://trello.com/c/FT01Ergd,Done,2020-06-18 17:45:26+00:00,2020-07-14 13:53:20+00:00,25,1
22,IIS,https://trello.com/c/SfIErnCj,Done,2020-06-03 15:15:40+00:00,2020-06-24 16:50:56+00:00,21,1
23,SECURELINK,https://trello.com/c/ctCn2KTX,Done,2020-05-27 17:46:16+00:00,2020-06-24 16:07:58+00:00,27,1
37,ADAUDIT_PLUS,https://trello.com/c/yD3G9Pmm,Done,2020-05-06 15:00:00+00:00,2020-05-27 18:00:00+00:00,21,1
38,VMWARE_ESX,https://trello.com/c/21kk3fio,Done,2020-04-28 15:00:00+00:00,2020-05-20 15:00:00+00:00,22,1
43,CLEARPASS,https://trello.com/c/TqKPkS6V,Done,2020-03-05 15:00:00+00:00,2020-04-03 15:00:00+00:00,29,1


In [357]:
done["duration"].describe()

count    14.000000
mean      2.500000
std       2.345208
min       0.000000
25%       1.000000
50%       1.000000
75%       4.750000
max       6.000000
Name: duration, dtype: float64

In [354]:
done.set_index("end_time")["duration"].resample("M").mean()

end_time
2020-03-31 00:00:00+00:00    12.500000
2020-04-30 00:00:00+00:00    19.666667
2020-05-31 00:00:00+00:00    22.166667
2020-06-30 00:00:00+00:00    17.375000
2020-07-31 00:00:00+00:00    17.400000
2020-08-31 00:00:00+00:00    20.500000
2020-09-30 00:00:00+00:00    23.800000
Freq: M, Name: duration, dtype: float64

In [355]:
done.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 45
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   name        31 non-null     object             
 1   shortUrl    31 non-null     object             
 2   idList      31 non-null     object             
 3   start_time  31 non-null     datetime64[ns, UTC]
 4   end_time    31 non-null     datetime64[ns, UTC]
 5   duration    31 non-null     int64              
 6   count       31 non-null     int64              
dtypes: datetime64[ns, UTC](2), int64(2), object(3)
memory usage: 1.9+ KB
